In [14]:
from GLMM_LA import *

In [43]:
import os
import sys

file_dir = '../../Covid_data_GLMM/Distributed/'
file_names = os.listdir(file_dir)

In [44]:
file_names

['mid_west.csv', 'others.csv', 'north_east.csv', 'south.csv', 'west.csv']

In [90]:
def LA(X, y, intercept=False):
    # Added regularization
    pre_score = -10**10
    # Number of variables
    p = X[0].shape[1]
    for lam in np.arange(0, 3, 1):
        try:
            # Regression with regularization
            beta = np.repeat(0, p).reshape(p, 1)
            mu = np.repeat(0.1, len(y))
            tau = 1

            start_time = time.time()
            # print('Initial beta:', beta, "\n")
            for step_mu in range(3):
                for i in range(len(mu)):
                    mu[i] = max_mu(X[i], y[i], mu[i], beta, tau)
                for step in range(50):
                    l1 = 0
                    l2 = 0
                    for i in range(len(mu)):
                        l1 += l_1(X[i], y[i], mu[i], beta, tau)
                        l2 += l_2(X[i], y[i], mu[i], beta, tau)
                    l1 -= (2 * lam * beta.transpose())[0]
                    l2 -= np.diag(np.repeat(2 * lam, p))
                    delta = l1 @ inv(l2)
                    new_beta = beta - delta.reshape(p, 1)
                    if max(np.abs(delta)) < 10 ** (-6):
                        break;
                    if max(np.abs(delta)) > 10 ** (3):
                        break;
                    beta = new_beta
                    if True in np.isnan(beta):
                        break;
#                     print('Step ', step + 1, ':\n')
#                     print('Beta:\n', beta, '\n')
#                     print('Diff:\n', delta, '\n')
#                     print('Lam:\n', lam, '\n')

                if True in np.isnan(beta):
                    break;
            # print('Beta:\n', beta, '\n')
            # print("--- %s seconds ---" % (time.time() - start_time))
            score = 0
            for i in range(len(X)):
                score += l(X[i], y[i], mu[i], beta, tau) - sum(lam * (beta) **2)
            print('score:\n',score)
            if score > pre_score:
                optimized_beta = beta
                optimized_mu = mu
                optimized_lam = lam
                # reset
                pre_score = score
        except:
            continue;
    return [optimized_beta, optimized_mu]

# two sites test

In [64]:
df1 = pd.read_csv(file_dir + 'mid_west.csv', index_col=0)
df2 = pd.read_csv(file_dir + 'north_east.csv', index_col=0)

In [65]:
df1.insert(loc=0, column='Intercept', value=1)

In [66]:
df2.insert(loc=0, column='Intercept', value=1)

In [68]:
data1 = np.array(df1.drop('target', axis=1))
out1 = np.array(df1.target).reshape(len(df1),1)
data2 = np.array(df2.drop('target', axis=1))
out2 = np.array(df2.target).reshape(len(df2),1)

In [69]:
data = [data1, data2]
out = [out1, out2]

In [91]:
[beta, mu] = LA(data, out)

score:
 [nan]
score:
 [-13694.99969362]


In [71]:
beta

array([[-2.88321844],
       [ 0.03987302],
       [ 0.40168388],
       [ 0.23657239],
       [ 0.22284408],
       [ 0.24693225],
       [-0.26007904],
       [ 0.03549004],
       [ 0.09137029],
       [ 0.1430452 ],
       [ 0.05868758],
       [ 0.26587798],
       [ 0.25562317],
       [-0.17197799],
       [ 0.13747912],
       [ 0.45540237],
       [ 0.08442922],
       [-0.13383609],
       [ 0.15650094]])

In [40]:
data1

array([[60,  0,  0, ...,  1,  0,  1],
       [58,  1,  1, ...,  0,  0,  1],
       [68,  0,  0, ...,  0,  0,  1],
       ...,
       [65,  1,  0, ...,  0,  0,  1],
       [59,  1,  0, ...,  1,  1,  0],
       [54,  1,  1, ...,  1,  0,  1]])

In [76]:
var_name = df1.drop('target', axis=1).columns

In [79]:
def output(X, beta, t, var_name):
  # Number of variables
    p = X[0].shape[1]

    X = np.concatenate(X)

    V = np.diagflat(Pi(X, beta, 0) * (1 - Pi(X, beta, 0)))

    SE = np.sqrt(np.diag(inv(np.transpose(X) @ V @ X))).reshape(p,1)

    Z = beta/SE

    P = 2 * norm.cdf(-1 * np.abs(Z))

    CI_025  = beta - 1.959964 * SE
    CI_975  = beta + 1.959964 * SE

    df = pd.DataFrame({'Coef': np.transpose(beta)[0], 'Std.Err': np.transpose(SE)[0],
                       'z': np.transpose(Z)[0], 'P-value': np.transpose(P)[0],
                       '[0.025': np.transpose(CI_025)[0], '0.975]': np.transpose(CI_975)[0],
                       'RunTime': t},
                      index = var_name)
    return df

In [81]:
output(data, beta, 10, var_name).to_csv('../../Covid_data_GLMM/Result_LA/')

IsADirectoryError: [Errno 21] Is a directory: '../../Covid_data_GLMM/Result_LA/'

In [36]:
df1.shape

(16546, 19)

# All

In [87]:
# from GLMM_LA import *
##################### Simulation with Covid Data ############################
import os
import sys

file_dir = '../../Covid_data_GLMM/Distributed/'
file_names = os.listdir(file_dir)
start_time = time.time()
data = []
out = []
for f in file_names:
	df = pd.read_csv(file_dir + f, index_col=0)
	# Add an intercept
	df.insert(loc=0, column='Intercept', value=1)
	data_ans = np.array(df.drop('target', axis=1))
	out_ans = np.array(df.target).reshape(len(df),1)
	data.append(data_ans)
	out.append(out_ans)
var_name = df.drop('target', axis=1).columns

# Regression
[beta, mu] = LA(data, out)
t = time.time() - start_time
output(data, beta, t, var_name)

Step  1 :

Beta:
 [[-4.03835258]
 [ 0.02977934]
 [ 0.28887582]
 [ 0.32153463]
 [ 0.16928221]
 [ 0.33706306]
 [-0.13256375]
 [-0.06332137]
 [ 0.09668389]
 [ 0.25926665]
 [ 0.0916968 ]
 [-0.13512826]
 [ 0.11083396]
 [-0.01327826]
 [-0.04945187]
 [ 0.38573054]
 [ 0.09867687]
 [-0.1685084 ]
 [ 0.10383092]] 

Diff:
 [ 4.03835258 -0.02977934 -0.28887582 -0.32153463 -0.16928221 -0.33706306
  0.13256375  0.06332137 -0.09668389 -0.25926665 -0.0916968   0.13512826
 -0.11083396  0.01327826  0.04945187 -0.38573054 -0.09867687  0.1685084
 -0.10383092] 

Lam:
 0 

Step  2 :

Beta:
 [[-18.87297171]
 [  0.04254637]
 [  0.58242545]
 [  0.73903121]
 [  0.45812387]
 [  0.7482742 ]
 [ -0.24361073]
 [ -0.13749891]
 [  0.27520068]
 [  1.03154647]
 [  0.44457609]
 [ -0.45407345]
 [  0.23988292]
 [ -0.05811066]
 [ -0.5640589 ]
 [  1.05476992]
 [  0.1271458 ]
 [ -0.44889001]
 [  0.04913019]] 

Diff:
 [ 1.48346191e+01 -1.27670325e-02 -2.93549630e-01 -4.17496579e-01
 -2.88841662e-01 -4.11211134e-01  1.11046976e-

/Users/wli17/Documents/GLMM/GLMM/Covid_LA/GLMM_LA.py:12: RuntimeWarning: overflow encountered in exp
  result = np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))))
/Users/wli17/Documents/GLMM/GLMM/Covid_LA/GLMM_LA.py:12: RuntimeWarning: invalid value encountered in true_divide
  result = np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))))
/Users/wli17/Documents/GLMM/GLMM/Covid_LA/GLMM_LA.py:13: RuntimeWarning: overflow encountered in exp
  if np.exp(x @ beta_0 + mu).max() == np.inf:
/Users/wli17/Documents/GLMM/GLMM/Covid_LA/GLMM_LA.py:30: RuntimeWarning: overflow encountered in exp
  result = np.nan_to_num(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu)) ** 2)), nan=0)
/Users/wli17/Documents/GLMM/GLMM/Covid_LA/GLMM_LA.py:30: RuntimeWarning: invalid value encountered in true_divide
  result = np.nan_to_num(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu)) ** 2)), nan=0)
/Users/wli17/Documents/GLMM/GLMM/Covid_LA

UnboundLocalError: local variable 'optimized_beta' referenced before assignment

In [85]:
pd.DataFrame(mu)

,0
0,-2.601864
1,-2.683113
2,-2.602924
3,-2.427379
4,-2.339420


In [129]:
from numba import cuda
from numpy.polynomial.hermite import hermgauss
from matplotlib import pyplot as plt
from numpy.linalg import inv
import pandas as pd
import numpy as np
import scipy
import time
from scipy.stats import norm

def Pi(x, beta_0, mu):
    return np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))))

def logsumexp(x):
    c = x.max()
    LSE = c + np.log(np.sum(np.exp(x - c)))
    return np.exp(x - LSE)

def g(x, y, mu, beta_0, tau=1):
    g = sum(y * logsumexp(Pi(x, beta_0, mu)) + (1 - y) * logsumexp(1 - Pi(x, beta_0, mu))) \
    + np.log((np.sqrt(2 * np.pi) * tau)**(-1) * np.exp(-mu**2/(2 * tau**2)))
    return g

def g_b(x, y, mu, beta_0, tau = 1):
    return np.sum(x * y - x * Pi(x, beta_0, mu), axis = 0)

def g_u(x, y, mu, beta_0, tau = 1):
    return sum(y - Pi(x, beta_0, mu)) - mu/tau**2

def g_uu(x, y, mu, beta_0, tau = 1):
    return sum(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))**2))) - 1/tau**2

def g_ub(x, y, mu, beta_0, tau = 1):
    return np.sum(- x * np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))**2)), axis = 0)

def g_bb(x, y, mu, beta_0, tau = 1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
        * (np.exp(x[i] @ beta_0 + mu) / (1 + np.exp(x[i] @ beta_0 + mu))**2))
    return result

def g_uuu(x, y, mu, beta_0, tau = 1):
    return sum(- np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)))

def g_uub(x, y, mu, beta_0, tau = 1):
    return np.sum(- x * np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)), axis = 0)

def g_ubb(x, y, mu, beta_0, tau = 1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
                         * (np.exp(x[i] @ beta_0 + mu) * (np.exp(x[i] @ beta_0 + mu) - 1)\
                                       / (1 + np.exp(x[i] @ beta_0 + mu))**3))
    return result

def g_uuuu(x, y, mu, beta_0, tau = 1):
    result = sum(- (np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)\
                 + (3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**4)\
                 - (np.exp(2 * (x @ beta_0 + mu))  / (1 + np.exp(x @ beta_0 + mu))**3)\
                )
    return result

def g_uuub(x, y, mu, beta_0, tau = 1):
    result = np.sum(- x * np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)\
                 + (3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**4)\
                 - (np.exp(2 * (x @ beta_0 + mu))  / (1 + np.exp(x @ beta_0 + mu))**3)), axis = 0)
    return result

def g_uubb(x, y, mu, beta_0, tau = 1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
                             * ((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)\
                 + (3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**4)\
                 - (np.exp(2 * (x @ beta_0 + mu))  / (1 + np.exp(x @ beta_0 + mu))**3)))
        
    return result

def omega(x, y, mu, beta_0, tau = 1):
    return np.sqrt(-1/g_uu(x, y, mu, beta_0))

def omega_b(x, y, mu, beta_0, tau = 1):
    return 0.5 * omega(x, y, mu, beta_0, tau)**3 * (g_uuu(x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau)\
                                                    + g_uub(x, y, mu, beta_0, tau))

def mu_b(x, y, mu, beta_0, tau = 1):
    return omega(x, y, mu, beta_0, tau)**2 * g_ub(x, y, mu, beta_0, tau)

def mu_bb(x, y, mu, beta_0, tau = 1):
    result = omega(x, y, mu, beta_0, tau)**2 * (mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
                                              @ mu_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
                                              * g_uuu(x, y, mu, beta_0, tau)\
                                              + 2 * mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
                                              @ g_uub(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
                                              + g_ubb(x, y, mu, beta_0, tau))
    return result

def omega_bb(x, y, mu, beta_0, tau = 1):
    result = 3/4 * omega(x, y, mu, beta_0, tau)**5 * (mu_b(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau)\
                                                      + g_uub(x, y, mu, beta_0, tau)).reshape(x.shape[1],1)\
     @ (mu_b(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau)\
        + g_uub(x, y, mu, beta_0, tau)).reshape(1,x.shape[1]) + 1/2 * omega(x, y, mu, beta_0, tau)**3\
     * (mu_bb(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau)\
        + (mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
           @ mu_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1]) * g_uuuu(x, y, mu, beta_0, tau))\
        + mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1) @ g_uuub(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
        + mu_bb(x, y, mu, beta_0, tau)
       )
    return result

# def f_k(k, x, y, mu, beta_0, tau = 1):
#     [x_k, h_k] = hermgauss(k)
#     return h_k * np.exp(g(x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0) + x_k**2)

# Plugged in log-sum-exp trick
def logsumexp(x):
    c = x.max()
    return c + np.log(np.sum(np.exp(x - c)))

def f_k(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
#     h_k = np.exp(-x_k**2)
    inside = g(x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0) + x_k**2
    result = np.exp(inside - logsumexp(inside))
    return result

# def l(k, x, y, mu, beta_0, tau = 1):
#     [x_k, h_k] = hermgauss(k)
#     return 0.5 * np.log(2 * np.pi) + np.log(omega(x, y, mu, beta_0, tau)) + np.log(sum(f_k(k, x, y, mu, beta_0, tau)))

def l(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    result = 0.5 * np.log(2 * np.pi) + np.log(omega(x, y, mu, beta_0, tau)) + np.log(sum(f_k(k, x, y, mu, beta_0, tau)))
    return result.astype(np.float64)

def f_k_b(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    result = 0
    for i in range(k):
        result += f_k(k, x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0, tau)[i] * g_b(x, y, mu + np.sqrt(2 * np.pi)\
                                                      * omega(x, y, mu, beta_0, tau) * x_k[i], beta_0, tau)
    return result

def f_k_u(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    return f_k(k, x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0, tau) * g_u(x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0, tau)

def f_k_w(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    return f_k(k, x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0, tau) * g_u(x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0, tau) * np.sqrt(2 * np.pi) * x_k

def l_1(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    result = omega_b(x, y, mu, beta_0, tau) / omega(x, y, mu, beta_0, tau)
    a = 1 / sum(f_k(k, x, y, mu, beta_0, tau))
    b = 0
    for i in range(k):
        b += f_k_u(k, x, y, mu, beta_0, tau)[i] * mu_b(x, y, mu, beta_0, tau)\
         + f_k_w(k, x, y, mu, beta_0, tau)[i] * omega_b(x, y, mu, beta_0, tau)
    b = b + f_k_b(k, x, y, mu, beta_0, tau)
    result = result + a * b
    return result

def f_k_ub(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    a = 0
    for i in range(k):
        a += f_k_u(k, x, y, mu, beta_0, tau)[i] * g_ub(x, y, mu, beta_0, tau)
    b = 0
    for i in range(k):
        b += f_k_u(k, x, y, mu, beta_0, tau)[i] * mu_b(x, y, mu, beta_0, tau)\
         + f_k_w(k, x, y, mu, beta_0, tau)[i] * omega_b(x, y, mu, beta_0, tau)
    b = b + f_k_b(k, x, y, mu, beta_0, tau)
    result = b * g_u(x, y, mu, beta_0, tau) + a
    return result

def f_k_wb(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    a = 0
    for i in range(k):
        a += f_k_u(k, x, y, mu, beta_0, tau)[i] * g_ub(x, y, mu, beta_0, tau) * x_k[i]
    b = 0
    for i in range(k):
        b += (f_k_u(k, x, y, mu, beta_0, tau)[i] * mu_b(x, y, mu, beta_0, tau)\
         + f_k_w(k, x, y, mu, beta_0, tau)[i] * omega_b(x, y, mu, beta_0, tau)\
         + f_k(k, x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau)\
               * x_k, beta_0, tau)[i] * g_b(x, y, mu + np.sqrt(2 * np.pi)\
                                                      * omega(x, y, mu, beta_0, tau) * x_k[i], beta_0, tau)) * x_k[i]
    result = np.sqrt(2 * np.pi) * (a + b)
    return result

def f_k_bb(k, x, y, mu, beta_0, tau = 1):
    result = f_k_b(k, x, y, mu, beta_0, tau).reshape(len(beta_0),1) @ g_b(x, y, mu, beta_0, tau).reshape(1,len(beta_0))\
    + sum(f_k(k, x, y, mu, beta_0, tau)) * g_bb(x, y, mu, beta_0, tau)
    return result

def part_1(k, x, y, mu, beta_0, tau = 1):
    result = f_k_ub(k, x, y, mu, beta_0, tau).reshape(len(beta_0),1)\
    @ mu_b(x, y, mu, beta_0, tau).reshape(1,len(beta_0))\
    + sum(f_k_u(k, x, y, mu, beta_0, tau)) * mu_bb(x, y, mu, beta_0, tau)\
    + f_k_wb(k, x, y, mu, beta_0, tau).reshape(len(beta_0),1)\
    @ omega_b(x, y, mu, beta_0, tau).reshape(1,len(beta_0))\
    + sum(f_k_w(k, x, y, mu, beta_0, tau)) * omega_bb(x, y, mu, beta_0, tau)\
    + f_k_bb(k, x, y, mu, beta_0, tau)
    return result


def l_2(k, x, y, mu, beta_0, tau = 1):
    a = 1 / sum(f_k(k, x, y, mu, beta_0, tau))
    b = 0
    for i in range(k):
        b += f_k_u(k, x, y, mu, beta_0, tau)[i] * mu_b(x, y, mu, beta_0, tau)\
         + f_k_w(k, x, y, mu, beta_0, tau)[i] * omega_b(x, y, mu, beta_0, tau)
    b = b + f_k_b(k, x, y, mu, beta_0, tau)
    part_2 = a * b
    l2 = omega(x, y, mu, beta_0, tau)**(-2) * (omega_bb(x, y, mu, beta_0, tau) * omega(x, y, mu, beta_0, tau)\
                                               - omega_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
                                               @ omega_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1]))\
     + sum(f_k(k, x, y, mu, beta_0, tau))**(-1) * part_1(k, x, y, mu, beta_0, tau) - part_2.reshape(x.shape[1],1)\
     @ part_2.reshape(1,x.shape[1])
    return l2

def max_mu(x, y, mu, beta_0, tau=1, max_iter=100):
    for step in range(max_iter):
#         print('Step: ', step, '\n')
        mu_new = mu - g_u(x, y, mu, beta_0, tau)/g_uu(x, y, mu, beta_0, tau)
        diff = mu_new - mu
#         print(diff)
        if np.abs(diff) < 10**(-10):
#             print(mu)
            break;
        mu = mu_new
    return mu

def mapping(x):
    return (x - min(x))/(max(x) - min(x))


In [102]:
# Definitions for GH
def GH(k, X, y):
    # Added regularization
    pre_score = -10**10
    # Number of variables
    p = X[0].shape[1]
    for lam in np.arange(0, 5, 1):
        try:
    
            # Regression with regularization
            beta = np.repeat(0, p).reshape(p, 1)
            mu = np.repeat(0.1, len(y))
            tau = 1

            start_time = time.time()
            # print('Initial beta:', beta, "\n")
            for step_mu in range(3):
                for i in range(len(mu)):
                    mu[i] = max_mu(X[i], y[i], mu[i], beta, tau)
                for step in range(20):
                    l1 = 0
                    l2 = 0
                    for i in range(len(mu)):
                        l1 += l_1(k, X[i], y[i], mu[i], beta, tau)
                        l2 += l_2(k, X[i], y[i], mu[i], beta, tau)
                    l1 -= (2 * lam * beta.transpose())[0]
                    l2 -= np.diag(np.repeat(2 * lam, p))
                    delta = l1 @ inv(l2)
                    new_beta = beta - delta.reshape(p, 1)
                    if max(np.abs(delta)) < 10 ** (-6):
                        break;
                    if max(np.abs(delta)) > 10 ** (3):
                        break;
                    beta = new_beta
                    if True in np.isnan(beta):
                        break;
                    print('Step ', step + 1, ':\n')
                    print('Beta:\n', beta, '\n')
                    print('Diff:\n', delta, '\n')
                    print('Lam:\n', lam, '\n')
                if True in np.isnan(beta):
                    break;
            # print('Beta:\n', beta, '\n')
            # print("--- %s seconds ---" % (time.time() - start_time))
            score = 0
            for i in range(len(X)):
                score += l(k, X[i], y[i], mu[i], beta, tau) - sum(lam * (beta) **2)
            print('score:\n', score)
            if score > pre_score:
                optimized_beta = beta
                optimized_mu = mu
                optimized_lam = lam
                # reset pre_score
                pre_score = score
                optimized_score = score
        except:
            continue;
    return [optimized_beta, optimized_mu, optimized_lam, optimized_score]

def output(X, beta, t, var_name):
    # Number of variables
    p = X[0].shape[1]

    X = np.concatenate(X)

    V = np.diagflat(Pi(X, beta, 0) * (1 - Pi(X, beta, 0)))

    SE = np.sqrt(np.diag(inv(np.transpose(X) @ V @ X))).reshape(p,1)

    Z = beta/SE

    P = 2 * norm.cdf(-1 * np.abs(Z))

    CI_025  = beta - 1.959964 * SE
    CI_975  = beta + 1.959964 * SE

    df = pd.DataFrame({'Coef': np.transpose(beta)[0], 'Std.Err': np.transpose(SE)[0],
                       'z': np.transpose(Z)[0], 'P-value': np.transpose(P)[0],
                       '[0.025': np.transpose(CI_025)[0], '0.975]': np.transpose(CI_975)[0],
                       'RunTime': t},
                      index = var_name)
    return df

In [103]:
GH(2, data, out)

Step  1 :

Beta:
 [[-2.41668409]
 [ 0.02961503]
 [ 0.27498578]
 [ 0.29667015]
 [ 0.18386811]
 [ 0.35801398]
 [-0.0214843 ]
 [ 0.00478358]
 [ 0.04965844]
 [ 0.29039869]
 [ 0.19294793]
 [-0.28692041]
 [ 0.1719659 ]
 [ 0.0241423 ]
 [-0.09880913]
 [ 0.38441172]
 [ 0.04927553]
 [-0.11127229]
 [ 0.11235901]] 

Diff:
 [ 2.41668409 -0.02961503 -0.27498578 -0.29667015 -0.18386811 -0.35801398
  0.0214843  -0.00478358 -0.04965844 -0.29039869 -0.19294793  0.28692041
 -0.1719659  -0.0241423   0.09880913 -0.38441172 -0.04927553  0.11127229
 -0.11235901] 

Lam:
 0 

Step  2 :

Beta:
 [[-3.13240091]
 [ 0.03779996]
 [ 0.34663734]
 [ 0.34512876]
 [ 0.21672208]
 [ 0.40087133]
 [-0.04891404]
 [ 0.01255881]
 [ 0.05245042]
 [ 0.19956524]
 [ 0.15936711]
 [-0.28155024]
 [ 0.17200241]
 [ 0.02238557]
 [-0.0111393 ]
 [ 0.34883038]
 [ 0.07402857]
 [-0.09296104]
 [ 0.18448432]] 

Diff:
 [ 7.15716821e-01 -8.18493068e-03 -7.16515579e-02 -4.84586034e-02
 -3.28539736e-02 -4.28573520e-02  2.74297347e-02 -7.77523403e-03

Step  15 :

Beta:
 [[-3.48773207]
 [ 0.04273866]
 [ 0.37329152]
 [ 0.36307995]
 [ 0.17821474]
 [ 0.39477702]
 [-0.16462491]
 [-0.07367349]
 [ 0.09179126]
 [ 0.15634762]
 [ 0.05847115]
 [-0.11857184]
 [ 0.1471448 ]
 [ 0.00649004]
 [ 0.09606023]
 [ 0.32752789]
 [ 0.14055623]
 [-0.15520585]
 [ 0.21288053]] 

Diff:
 [-5.03336847e-03  2.80136463e-05  7.60939392e-04  2.03790522e-04
  3.08538944e-03  1.74112276e-03  2.06797765e-03  2.45708130e-03
 -6.11068001e-04  5.04352979e-04  4.02535564e-03 -3.93991211e-03
  6.62749818e-04  6.80073884e-04 -2.19867116e-03 -4.77223466e-04
 -1.17964281e-03  1.97468955e-03  3.65222913e-04] 

Lam:
 0 

Step  16 :

Beta:
 [[-3.48330322]
 [ 0.04271318]
 [ 0.37263004]
 [ 0.36293577]
 [ 0.17546939]
 [ 0.39322307]
 [-0.16638119]
 [-0.07580363]
 [ 0.09229524]
 [ 0.1558972 ]
 [ 0.0548899 ]
 [-0.11519662]
 [ 0.14656759]
 [ 0.00587868]
 [ 0.09802784]
 [ 0.32800461]
 [ 0.14155382]
 [-0.15691351]
 [ 0.21256134]] 

Diff:
 [-4.42885330e-03  2.54831270e-05  6.61478888e-04  

UnboundLocalError: local variable 'optimized_beta' referenced before assignment

In [109]:
gh_result = pd.read_csv('/Users/wli17/Documents/GLMM/Covid_data_GLMM/Result_GH/result_gh.csv', index_col=0)

In [113]:
beta = gh_result.Coef

In [114]:
beta

Intercept                -3.476182
age                       0.042685
Gen_M                     0.369853
race_Asian                0.363802
race_Caucasian            0.155659
race_Other/Unknown        0.382970
ethnicity_Not Hispanic   -0.178002
ethnicity_Unknown        -0.091355
COPD_Y                    0.095744
CHF_Y                     0.153521
CKD_Y                     0.028346
MS_Y                     -0.093454
RA_Y                      0.143521
LU_Y                      0.000916
HTN_Y                     0.112989
IHD_Y                     0.334440
DIAB_Y                    0.148526
ASTH_Y                   -0.169361
Obese_Y                   0.211295
Name: Coef, dtype: float64

In [127]:
score = 0
for i in range(len(data)):
    score += l(2, data[i], out[i], mu[i], np.array(beta).reshape(len(beta),1)) - sum(2 * (np.array(beta).reshape(len(beta),1)) **2)

In [128]:
score

array([-140.08162535])

In [130]:
l(2, data[0], out[0], mu[0], np.array(beta).reshape(len(beta),1))

array([-2.61370761])

In [125]:
np.array(beta).reshape(len(beta),1)

array([[-3.47618244e+00],
       [ 4.26851403e-02],
       [ 3.69853034e-01],
       [ 3.63802296e-01],
       [ 1.55658826e-01],
       [ 3.82970188e-01],
       [-1.78001774e-01],
       [-9.13551560e-02],
       [ 9.57437303e-02],
       [ 1.53521451e-01],
       [ 2.83463428e-02],
       [-9.34535657e-02],
       [ 1.43520727e-01],
       [ 9.15975041e-04],
       [ 1.12988722e-01],
       [ 3.34440317e-01],
       [ 1.48526446e-01],
       [-1.69360868e-01],
       [ 2.11294517e-01]])